# Huggingface Sagemaker-sdk - Distributed Training Demo
### Distributed Token Classification (NER) with `transformers` scripts +  `Trainer` and `conll2003` dataset

- Source: https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/token-classification

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Development environment](#Development-environment)  
    3. [Permissions](#Permissions) 
4. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    3. [Attach to old training job to an estimator ](#Attach-to-old-training-job-to-an-estimator)  
5. [_Coming soon_:Push model to the Hugging Face hub](#Push-model-to-the-Hugging-Face-hub)

# Introduction

Welcome to our end-to-end `distributed` "Token Classification" (NER) example. In this demo, we will use the Hugging Face `transformers` together with a custom Amazon sagemaker-sdk extension to fine-tune a pre-trained transformer for token-classification on multiple-gpus. In particular, the pre-trained model will be fine-tuned using the `conll2003` dataset. The demo will use the new `smdistributed` library to run training on multiple gpus as training scripting we are going to use one of the `transformers` [example scripts from the repository](https://github.com/huggingface/transformers/blob/master/examples/question-answering/run_qa.py).

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on. 

_**NOTE: You can run this demo in Sagemaker Studio, your local machine or Sagemaker Notebook Instances**_

# Development Environment and Permissions 

## Installation

_*Note:* we only install the required libraries from Hugging Face and AWS. You also need PyTorch or Tensorflow, if you haven´t it installed_

In [1]:
!pip install "sagemaker>=2.48.0"  --upgrade -q

## Development environment 

In [2]:
import sagemaker.huggingface

## Permissions

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [3]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::110564771975:role/service-role/AmazonSageMaker-ExecutionRole-20210806T162946
sagemaker bucket: sagemaker-eu-west-2-110564771975
sagemaker session region: eu-west-2


# Fine-tuning & starting Sagemaker Training Job

In order to create a sagemaker training job we need an `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. In a Estimator we define, which fine-tuning script should be used as `entry_point`, which `instance_type` should be used, which `hyperparameters` are passed in .....

```python
hyperparameters = {
	'model_name_or_path':'philschmid/distilroberta-base-ner-conll2003',
	'output_dir':'/opt/ml/model',
    'epochs': 1,
    'train_batch_size': 32,
	# add your remaining hyperparameters
	# more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/token-classification
}

huggingface_estimator = HuggingFace(
    entry_point='run_ner.py',
    source_dir='./examples/pytorch/token-classification',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    git_config=git_config,
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    hyperparameters = hyperparameters
)
```

When we create a SageMaker training job, SageMaker takes care of starting and managing all the required ec2 instances for us with the `huggingface` container, uploads the provided fine-tuning script `train.py` and downloads the data from our `sagemaker_session_bucket` into the container at `/opt/ml/input/data`. Then, it starts the training job by running. 

```python
/opt/conda/bin/python train.py --epochs 1 --model_name philschmid/distilroberta-base-ner-conll2003 --train_batch_size 32
```

The `hyperparameters` you define in the `HuggingFace` estimator are passed in as named arguments. 

Sagemaker is providing useful properties about the training environment through various environment variables, including the following:

* `SM_MODEL_DIR`: A string that represents the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting.

* `SM_NUM_GPUS`: An integer representing the number of GPUs available to the host.

* `SM_CHANNEL_XXXX:` A string that represents the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the HuggingFace estimator’s fit call, named `train` and `test`, the environment variables `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` are set.


To run your training job locally you can define `instance_type='local'` or `instance_type='local_gpu'` for gpu usage. _Note: this does not working within SageMaker Studio_


## Load training data from `flair`, split into `train`/`dev`/`test`, and upload to S3
- https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md
- https://github.com/huggingface/transformers/blob/main/tests/fixtures/tests_samples/conll/sample.json
- https://github.com/huggingface/transformers/issues/8698
- https://huggingface.co/docs/datasets/loading#json-files

In [4]:
import flair
import inspect
import json
from os.path import exists as path_exists
from pathlib import Path

An example of what `train.json` should look like

```python
{"id": "1", "tokens": ["APPLICATION", "and", "Affidavit", "for", "Search", ...], "ner_tags": ["O", "O", "O", "O", "O", ...]}

...

{"id": "n", "tokens": ["APPLICATION", "and", "Affidavit", "for", "Search", ...], "ner_tags": ["O", "O", "O", "O", "O", ...]}
```

In [ ]:
def get_flair_corpus(dataset_name,dataset_arguments:str=""):
    ner_task_mapping = {}

    for name, obj in inspect.getmembers(flair.datasets.sequence_labeling):
        if inspect.isclass(obj):
            if name.startswith("NER") or name.startswith("CONLL") or name.startswith("WNUT"):
                ner_task_mapping[name] = obj

    dataset_args = {}

    if dataset_arguments:
        dataset_args = json.loads(dataset_arguments)

    if dataset_name not in ner_task_mapping:
        raise ValueError(f"Dataset name {dataset_name} is not a valid Flair datasets name!")

    return ner_task_mapping[dataset_name](**dataset_args)

In [219]:
def upload_flair_corpus_to_s3(corpus, filepath, key_prefix = 'distributed_conll2003_data'):
    if not path_exists('dataset'): !mkdir dataset
    path = f'dataset/{filepath}'
    
    lines = []
    for idx, sentence in enumerate(corpus):
        tokens = []
        ner_tags = []
        for token in sentence:
            try: tag = token.tag
            except: tag = "O"
            tokens.append(token.text)
            ner_tags.append(tag)
        lines.append({"tokens": (' '.join(tokens)).strip(), "ner_tags": (','.join(ner_tags)).strip()})
            
    # convert to a list of JSON strings
    json_lines = [json.dumps(l) for l in lines]

    # join lines and save to .json file
    json_data = '\n'.join(json_lines)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(json_data)
        
    # save to s3    
    s3_path = sess.upload_data(path, key_prefix=key_prefix)
    print('done! file also saved locally to "{}"'.format(filepath))
    
    return s3_path

In [28]:
corpus = get_flair_corpus(dataset_name='NER_ENGLISH_PERSON')
print(corpus)

2022-05-21 00:01:15,488 Reading data from /home/ec2-user/.flair/datasets/ner_english_person
2022-05-21 00:01:15,489 Train: /home/ec2-user/.flair/datasets/ner_english_person/bigFile.conll
2022-05-21 00:01:15,489 Dev: None
2022-05-21 00:01:15,490 Test: None
Corpus: 28362 train + 3151 dev + 3501 test sentences


In [29]:
tag_dictionary = corpus.make_label_dictionary("ner")
print(tag_dictionary)

2022-05-21 00:01:21,657 Computing label dictionary. Progress:


28362it [00:00, 71885.80it/s]

2022-05-21 00:01:22,054 Dictionary created for label 'ner' with 4 values: M (seen 27887 times), F (seen 4543 times), A (seen 634 times)
Dictionary with 4 tags: <unk>, M, F, A


In [220]:
train_input_path = upload_flair_corpus_to_s3(corpus.train, 'train.json')
test_input_path = upload_flair_corpus_to_s3(corpus.test, 'test.json')
valid_input_path = upload_flair_corpus_to_s3(corpus.dev, 'valid.json')

done! file also saved locally to "train.csv"
done! file also saved locally to "test.csv"
done! file also saved locally to "valid.csv"


## Creating an Estimator and start a training job

In this example we are going to use the capability to download/use a fine-tuning script from a `git`- repository. We are using the `run_ner.py` from the `transformers` example scripts. You can find the code [here](https://github.com/huggingface/transformers/tree/main/examples/pytorch/token-classification).

See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
https://huggingface.co/docs/datasets/loading_datasets.html.

In [12]:
from sagemaker.huggingface import HuggingFace

In [ ]:
data = {
    'train': train_input_path,
    'test': test_input_path,
    'valid': valid_input_path,
}

In [13]:
# hyperparameters, which are passed into the training job
hyperparameters={
    'model_name_or_path': 'philschmid/distilroberta-base-ner-conll2003',
    'train_file': "/opt/ml/input/data/train/train.json",
    'test_file': "/opt/ml/input/data/test/test.json",
    'validation_file': "/opt/ml/input/data/valid/valid.json",
    'learning_rate': 5e-05,
    'per_device_train_batch_size': 32,
    'per_device_eval_batch_size': 16,
    'seed': 42,
    'lr_scheduler_type': 'linear',
    'num_train_epochs': 50,
    'fp16': True,
    'output_dir': '/opt/ml/model',
    'do_train': True,
    'do_eval': True,
    'max_seq_length': 128,
}

# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# instance configurations
instance_type='ml.p3.2xlarge'
instance_count=1
volume_size=100

# metric definition to extract the results
metric_definitions=[
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {'Name': 'train_samples_per_second', 'Regex': "train_samples_per_second.*=\D*(.*?)$"},
    {'Name': 'epoch', 'Regex': "epoch.*=\D*(.*?)$"},
    {'Name': 'f1', 'Regex': "f1.*=\D*(.*?)$"},
    {'Name': 'precision', 'Regex': "precision.*=\D*(.*?)$"},
    {'Name': 'recall', 'Regex': "recall.*=\D*(.*?)$"},
    {'Name': 'exact_match', 'Regex': "exact_match.*=\D*(.*?)$"}]

In [14]:
# estimator
huggingface_estimator = HuggingFace(entry_point='run_ner.py',
                                    source_dir='./examples/pytorch/token-classification',
                                    git_config=git_config,
                                    metric_definitions=metric_definitions,
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.17.0',
                                    pytorch_version='1.10.2',
                                    py_version='py38',
#                                     distribution= distribution,
                                    hyperparameters = hyperparameters)

In [1]:
# starting the train job
huggingface_estimator.fit(data, wait=False)

## Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

Then, we use the returned predictor object to call the endpoint.

In [ ]:
predictor.predict({
	'inputs': "My name is Sarah Jessica Parker but you can call me Jessica"
})

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_endpoint()

## Estimator Parameters

In [ ]:
# container image used for training job
print(f"container image used for training job: \n{huggingface_estimator.image_uri}\n")

# s3 uri where the trained model is located
print(f"s3 uri where the trained model is located: \n{huggingface_estimator.model_data}\n")

# latest training job name for this estimator
print(f"latest training job name for this estimator: \n{huggingface_estimator.latest_training_job.name}\n")

In [ ]:
# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

## Attach an old training job to an estimator 

In Sagemaker you can attach an old training job to an estimator to continue training, get results etc..

In [ ]:
from sagemaker.estimator import Estimator

# job which is going to be attached to the estimator
# old_training_job_name=''

In [ ]:
# attach old training job
# huggingface_estimator_loaded = Estimator.attach(old_training_job_name)

# get model output s3 from training job
# huggingface_estimator_loaded.model_data

## References

- [Named-Entity-Recognition-on-HuggingFace](https://wandb.ai/biased-ai/Named-Entity%20Recognition%20on%20HuggingFace/reports/Named-Entity-Recognition-on-HuggingFace--Vmlldzo3NTk4NjY)
- [`run_ner.py` script in HuggingFace](https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py)
- [Named Entity Recognition with Transformers](https://chriskhanhtran.github.io/posts/named-entity-recognition-with-transformers/)
- [Token classification in HuggingFace](https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/token-classification)
- [Training Custom NER Model Using Flair](https://medium.com/thecyphy/training-custom-ner-model-using-flair-df1f9ea9c762)